<a href="https://colab.research.google.com/github/nourhanOfTerra/CIFAR-10/blob/main/CIFAR10_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CIFAR-10 Classification Model: Support Vector Machines

The overall guide: https://towardsdatascience.com/multiclass-classification-using-k-nearest-neighbours-ca5281a9ef76

##Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/CIFAR-10')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Importing the necessary libraries

###General Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

###Model Specific Libraris

In [3]:
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

##Importing the data from csv files to variables

In [4]:
extracted_features = pd.read_csv('extracted features.csv')
labels = pd.read_csv('labels.csv')

##Dividing the data into training, validation and tesing sets

In [5]:
standard_extracted_features = pd.DataFrame(StandardScaler().fit_transform(extracted_features))        # Necessary standardisation step
X_train, X_test, y_train, y_test = train_test_split(standard_extracted_features, labels, test_size = 0.2)

##Training and Cross Validation
Using GridSearchCV to optimize the hyperparameters, then the model is trained.

Guide: https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/

In [ ]:
parameters = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf', 'poly']}
model = GridSearchCV(svm.SVC(), parameters, refit = True, verbose = 3)
model.fit(X_train, y_train.values.ravel())
print(model.best_params_)
print()
print(model.best_estimator_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.101 total time=15.6min
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.101 total time=15.6min
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.101 total time=15.7min
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.101 total time=15.5min
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.101 total time=15.7min
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.390 total time=15.2min
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.392 total time=14.9min


Apparently, we can get the highest accuracy (25%) by putting the number of neighbors to be 1. The thing is, 1 Nearest Neighbor will cause overfitting of the model, and we don't want that now, do we? Every thing else will give nearly the same accuracy, so let's go with the conventional 5-NN to fit our model.

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

##Testing

In [ ]:
y_predicted = knn.predict(X_test)
confused = confusion_matrix(y_test, y_predicted)
print("The confusion matrix: ")
print(confused)
report = classification_report(y_test, y_predicted)
print("\nThe classification report: ")
print(report)
acc_score = accuracy_score(y_test, y_predicted)
print("\nTest Accuracy: ", acc_score)

The confusion matrix: 
[[594   0 119   8 100   0  12   1 394   0]
 [270  62 117  32 270   6  30   2 376   1]
 [286   0 340  28 395   7  22   0 105   0]
 [333   1 279 109 319  17  30   1 126   0]
 [277   0 137  16 658   1   9   0  91   0]
 [314   0 334  85 308  60  31   2  89   0]
 [182   2 192  47 533   1 117   1  89   0]
 [247   1 214  35 496  14  20  71 108   0]
 [316   0  71  13  87   2   7   0 743   0]
 [225   7 170  43 292   7  26   4 409   4]]

The classification report: 
              precision    recall  f1-score   support

           0       0.20      0.48      0.28      1228
           1       0.85      0.05      0.10      1166
           2       0.17      0.29      0.22      1183
           3       0.26      0.09      0.13      1215
           4       0.19      0.55      0.28      1189
           5       0.52      0.05      0.09      1223
           6       0.38      0.10      0.16      1164
           7       0.87      0.06      0.11      1206
           8       0.29      0